In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Individual Model Predictions**

In [35]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import chardet
import sys
from sklearn.metrics import accuracy_score

# ---- CUSTOM SELF-ATTENTION LAYER ----
from tensorflow.keras.layers import Layer
import tensorflow.keras.backend as K
import keras

@keras.saving.register_keras_serializable()
class SelfAttentionLayer(Layer):
    def __init__(self, D_att, **kwargs):
        super(SelfAttentionLayer, self).__init__(**kwargs)
        self.D_att = D_att

    def build(self, input_shape):
        self.Q = self.add_weight(
            name      = 'query_vector',
            shape     = (1, self.D_att),
            initializer = 'random_normal',
            trainable = True
        )
        super(SelfAttentionLayer, self).build(input_shape)

    def call(self, inputs):
        batch_size      = tf.shape(inputs)[0]
        sequence_length = tf.shape(inputs)[1]
        Q_expanded      = tf.tile(self.Q, [batch_size, sequence_length, 1])
        scores          = K.sum(Q_expanded * inputs, axis=-1)
        weights         = tf.keras.activations.softmax(scores)
        output          = K.sum(K.expand_dims(weights, -1) * inputs, axis=1)
        return output

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.D_att)

# ---- CONFIGURATION ----
DATASET_PATH  = '/content/drive/MyDrive/Synthetic Data/Final Fused Data.csv'
OUTPUT_PATH   = '/content/drive/MyDrive/Synthetic Data/Best Final Predictions.csv'
MODEL1_PATH   = '/content/drive/MyDrive/Thesis Models/mdd_lstm.h5'
MODEL2_PATH   = '/content/drive/MyDrive/Thesis Models/lstm_stress_dryad_model.h5'
MODEL3_PATH   = '/content/drive/MyDrive/Thesis Models/rnn_model.h5'

# Actual-label columns
ACTUAL_LABEL_COLS = {
    'mdd':    'Actual_mdd',
    'stress': 'Actual_Stress',
    'sphere': 'predicted_labels'
}

# 0-based start columns
MODEL_START_COLS = {
    'mdd':     0,
    'stress':  120,
    'sphere':  600
}

# (timesteps, features) for each
MODEL_SHAPES = {
    'mdd':     (30, 4),    # 120 cols
    'stress':  (10, 48),   # 480 cols
    'sphere':  (20, 23)    # 460 cols
}

# ---- 1. LOAD DATA ----
print(f"Reading dataset from {DATASET_PATH}...")
for enc in ('utf-8', 'utf-8-sig', 'latin1'):
    try:
        df = pd.read_csv(DATASET_PATH, encoding=enc)
        print(f"→ loaded with encoding={enc!r}, shape={df.shape}")
        break
    except Exception:
        continue
else:
    sys.exit("ERROR: Could not read CSV with utf-8 / utf-8-sig / latin1")

# ---- 2. LOAD MODELS ----
print("\nLoading models...")
try:
    model1 = load_model(MODEL1_PATH, custom_objects={'SelfAttentionLayer': SelfAttentionLayer})
    print("→ Model 1 loaded")
    model2 = load_model(MODEL2_PATH, custom_objects={'SelfAttentionLayer': SelfAttentionLayer})
    print("→ Model 2 loaded")
    model3 = load_model(MODEL3_PATH, custom_objects={'SelfAttentionLayer': SelfAttentionLayer})
    print("→ Model 3 loaded")
except Exception as e:
    sys.exit(f"ERROR loading models: {e}")

# ---- 3. PREDICT & ACCURACY ----
out = df.copy()
accuracies = {}

for name, model in [('mdd', model1), ('stress', model2), ('sphere', model3)]:
    ts, features = MODEL_SHAPES[name]
    flat_size = ts * features
    start = MODEL_START_COLS[name]
    actual_cols = flat_size  # Use full column set for all models
    end_actual = start + actual_cols

    print(f"\n→ Preparing inputs for '{name}' (using {flat_size} cols)")

    # Extract the full set of columns
    raw_actual = df.iloc[:, start:end_actual].values
    if raw_actual.shape[1] != flat_size:
        sys.exit(f"ERROR: '{name}' expected {flat_size} cols but got {raw_actual.shape[1]}")

    # Reshape to (n_samples, timesteps, features)
    X = raw_actual.reshape(-1, ts, features)
    print(f"   • reshaped to {X.shape}")

    # Predict
    preds = model.predict(X, batch_size=32, verbose=1)
    # To class indices if softmax
    if preds.ndim == 2 and preds.shape[1] > 1:
        labels = np.argmax(preds, axis=1)
    else:
        labels = preds.flatten().astype(int)

    # Remap '2'→'1' for stress
    if name == 'stress':
        labels = np.where(labels == 2, 1, labels)
        print("   • remapped stress ‘2’→’1’")

    # Attach to DataFrame
    col_pred = {
        'mdd':    'Predicted_mdd',
        'stress': 'Predicted_Stress',
        'sphere': 'SPHERE Labels'
    }[name]
    out[col_pred] = labels

    # Accuracy
    actual_col = ACTUAL_LABEL_COLS[name]
    if actual_col in df:
        acc = accuracy_score(df[actual_col], labels)
        accuracies[name] = acc
        print(f"   • {name} accuracy: {acc:.4%}")
    else:
        accuracies[name] = None
        print(f"   • no '{actual_col}' column; skipped accuracy")

# ---- 4. SAVE & REPORT ----
out.to_csv(OUTPUT_PATH, index=False)

# ---- 4. SAVE & REPORT ----

# ⬇️ **SAVE ALL PREDICTIONS** ⬇️
out.to_csv(OUTPUT_PATH, index=False)
print(f"✅ All model predictions saved to: {OUTPUT_PATH}")

# (Optional) print out the first few rows of predictions
print(out[['Predicted_mdd', 'Predicted_Stress', 'SPHERE Labels']].head())

Reading dataset from /content/drive/MyDrive/Synthetic Data/Final Fused Data.csv...


→ loaded with encoding='utf-8', shape=(6897, 1065)

Loading models...
→ Model 1 loaded


→ Model 2 loaded
→ Model 3 loaded

→ Preparing inputs for 'mdd' (using 120 cols)
   • reshaped to (6897, 30, 4)
216/216 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step
   • mdd accuracy: 90.8946%

→ Preparing inputs for 'stress' (using 480 cols)
   • reshaped to (6897, 10, 48)
216/216 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
   • remapped stress ‘2’→’1’
   • stress accuracy: 55.2414%

→ Preparing inputs for 'sphere' (using 460 cols)
   • reshaped to (6897, 20, 23)
216/216 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step
   • sphere accuracy: 89.3577%
✅ All model predictions saved to: /content/drive/MyDrive/Synthetic Data/Best Final Predictions.csv
   Predicted_mdd  Predicted_Stress  SPHERE Labels
0              1                 1              8
1              1                 1              8
2              1                 1             10
3              1                 1             10
4              1                 1              8


# **Simple Attention Fusion**

In [64]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Softmax, Multiply, Lambda
import tensorflow.keras.backend as K
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# ---- 1. LOAD CSV ----
df = pd.read_csv('/content/drive/MyDrive/Synthetic Data/Best Final Predictions.csv')

# ---- 2. EXTRACT & ONE‑HOT ENCODE EACH PREDICTION SERIES ----
m1 = df['Predicted_mdd'].astype(int).values
m2 = df['Predicted_Stress'].astype(int).values
m3 = df['SPHERE Labels'].astype(int).values

# Compute number of classes as the max label+1 across all three
n_classes = max(m1.max(), m2.max(), m3.max()) + 1

P1 = to_categorical(m1, num_classes=n_classes)   # shape (N, C)
P2 = to_categorical(m2, num_classes=n_classes)   # shape (N, C)
P3 = to_categorical(m3, num_classes=n_classes)   # shape (N, C)

# ---- 3. STACK INTO META‑INPUT ----
# axis=1 will give shape (N, 3, C)
X_meta = np.stack([P1, P2, P3], axis=1)

# If you have a separate ground‑truth column, say `Final_Truth`, load and one‑hot that too:
y = df['predicted_labels'].astype(int).values
y_meta = to_categorical(y, num_classes=n_classes)

# ---- 4. SPLIT TRAIN/VAL ----
X_tr, X_val, y_tr, y_val = train_test_split(
    X_meta, y_meta, test_size=0.2, random_state=42, stratify=y
)

# ---- 5. BUILD ATTENTION‑FUSION MODEL ----
inp = Input(shape=(3, n_classes), name='meta_input')
# score each slot → (None,3,1)
scores = Dense(1, name='att_score')(inp)
# squeeze out the last dim → (None,3)
scores = Lambda(lambda x: K.squeeze(x, -1), name='att_squeeze')(scores)
# normalize across the 3 models → (None,3)
weights = Softmax(axis=1, name='att_softmax')(scores)
# expand back → (None,3,1)
weights_exp = Lambda(lambda x: K.expand_dims(x, -1), name='att_expand')(weights)
# weight the prob vectors → (None,3,C)
weighted = Multiply(name='att_mul')([inp, weights_exp])

# sum across the 3 models → (None,C)
fused_vec = Lambda(
    lambda x: K.sum(x, axis=1),
    output_shape=lambda input_shape: (input_shape[0], input_shape[2]),
    name='att_fused'
)(weighted)

# optional final Dense for refined softmax
out = Dense(n_classes, activation='softmax', name='final_softmax')(fused_vec)

fusion_model = Model(inputs=inp, outputs=out, name='attention_fusion')
fusion_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
fusion_model.summary()

# ---- 6. TRAIN ----
fusion_model.fit(
    X_tr, y_tr,
    validation_data=(X_val, y_val),
    epochs=30,
    batch_size=16,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True)]
)

# ---- 7. SAVE ----
fusion_model.save('/content/attention_fusion_from_preds.h5')
print("Saved fused attention‐model to disk.")

Model: "attention_fusion"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ meta_input          │ (None, 3, 20)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ att_score (Dense)   │ (None, 3, 1)      │         21 │ meta_input[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ att_squeeze         │ (None, 3)         │          0 │ att_score[0][0]   │
│ (Lambda)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ att_softmax         │ (None, 3)         │          0 │ att_squeeze[0][0] │
│ (Softmax)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ att_expand (Lambda) │ (None, 3, 1)      │          0 │ att_softmax[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ att_mul (Multiply)  │ (None, 3, 20)     │          0 │ meta_input[0][0], │
│                     │                   │            │ att_expand[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ att_fused (Lambda)  │ (None, 20)        │          0 │ att_mul[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ final_softmax       │ (None, 20)        │        420 │ att_fused[0][0]   │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 441 (1.72 KB)

 Trainable params: 441 (1.72 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/30
345/345 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.3003 - loss: 2.8178 - val_accuracy: 0.6775 - val_loss: 2.2948
Epoch 2/30
345/345 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6980 - loss: 2.1391 - val_accuracy: 0.7500 - val_loss: 1.7064
Epoch 3/30
345/345 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7519 - loss: 1.5854 - val_accuracy: 0.8152 - val_loss: 1.2262
Epoch 4/30
345/345 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8150 - loss: 1.1466 - val_accuracy: 0.8341 - val_loss: 0.9653
Epoch 5/30
345/345 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8395 - loss: 0.9221 - val_accuracy: 0.8442 - val_loss: 0.8338
Epoch 6/30
345/345 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8480 - loss: 0.8098 - val_accuracy: 0.8442 - val_loss: 0.7587
Epoch 7/30
345/345 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8510 - loss: 0.7303 - val_accuracy: 0.8435 - val_loss: 0.7102
Epoch 8/30
345/345 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8515 - loss: 0.6871 - val_accuracy: 0.

Saved fused attention‐model to disk.


# **Fusion Model Testing**

In [59]:
import tensorflow.keras.backend as K

def squeeze_fn(x):
    return K.squeeze(x, axis=-1)

def expand_fn(x):
    return K.expand_dims(x, axis=-1)

def fused_fn(x):
    return K.sum(x, axis=1)

# 1. Squeeze layer: (None, 3, 1) → (None, 3)
scores = Lambda(
    squeeze_fn,
    output_shape=lambda s: (s[0], s[1]),  # s = (batch, 3, 1) → (batch, 3)
    name="att_squeeze"
)(scores)

# 2. Expand dims: (None, 3) → (None, 3, 1)
weights_exp = Lambda(
    expand_fn,
    output_shape=lambda s: (s[0], s[1], 1),  # payload dims = 1
    name="att_expand"
)(weights)

# 3. Fuse: (None, 3, 1) after weighted → (None, 1)
fused_vec = Lambda(
    fused_fn,
    output_shape=lambda s: (s[0], s[2]),     # now s = (batch, 3, 1), so s[2] = 1
    name="att_fused"
)(weighted)

In [60]:
fusion_model.save('attention_fusion_from_preds_2.h5')

In [66]:
fusion_model.save_weights("fusion_weights.weights.h5")

In [67]:
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Input, Dense, Softmax, Multiply, Lambda
from tensorflow.keras.models import Model

# Named functions for Lambda layers
def squeeze_fn(x): return K.squeeze(x, axis=-1)
def expand_fn(x): return K.expand_dims(x, axis=-1)
def fused_fn(x): return K.sum(x, axis=1)

n_classes = 20  # Replace with your actual number of classes

# Build architecture
inp = Input(shape=(3, n_classes), name="meta_input")
scores = Dense(1, name="att_score")(inp)
scores = Lambda(
    squeeze_fn,
    output_shape=lambda s: (s[0], s[1]),  # (batch, 3)
    name="att_squeeze"
)(scores)
weights = Softmax(axis=1, name="att_softmax")(scores)
weights_exp = Lambda(
    expand_fn,
    output_shape=lambda s: (s[0], s[1], 1),
    name="att_expand"
)(weights)
weighted = Multiply(name="att_mul")([inp, weights_exp])
fused_vec = Lambda(
    fused_fn,
    output_shape=lambda s: (s[0], s[2]),
    name="att_fused"
)(weighted)
out = Dense(n_classes, activation='softmax', name="final_softmax")(fused_vec)

fusion_model = Model(inputs=inp, outputs=out, name="attention_fusion")

In [68]:
fusion_model.load_weights("/content/fusion_weights.weights.h5")

In [72]:
df_new = pd.read_csv("/content/drive/MyDrive/Synthetic Data/Final Fused Data.csv")

def slice_input(df, start, timesteps, features):
    flat = timesteps * features
    arr = df.iloc[:, start:start+flat].values
    return arr.reshape(-1, timesteps, features)

X1_new = slice_input(df_new, start=0,   timesteps=30, features=4)
X2_new = slice_input(df_new, start=120, timesteps=10, features=48)
X3_new = slice_input(df_new, start=600, timesteps=20, features=23)

P1_new = model1.predict(X1_new)
P2_new = model2.predict(X2_new)
P3_new = model3.predict(X3_new)

print("P1_new shape:", P1_new.shape)
print("P2_new shape:", P2_new.shape)
print("P3_new shape:", P3_new.shape)

216/216 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step
216/216 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
216/216 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step
P1_new shape: (6897, 2)
P2_new shape: (6897, 3)
P3_new shape: (6897, 20)


In [73]:
def pad_to(arr, target_dim):
    n, d = arr.shape
    if d < target_dim:
        pad = np.zeros((n, target_dim - d))
        return np.concatenate([arr, pad], axis=1)
    return arr

target = max(P1_new.shape[1], P2_new.shape[1], P3_new.shape[1])
P1_new = pad_to(P1_new, target)
P2_new = pad_to(P2_new, target)
P3_new = pad_to(P3_new, target)

In [74]:
import numpy as np
X_meta_new = np.stack([P1_new, P2_new, P3_new], axis=1)
# shape = (num_samples, 3, num_classes)
final_probs = fusion_model.predict(X_meta_new)
final_labels = final_probs.argmax(axis=1)

216/216 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step


In [75]:
probs = fusion_model.predict(X_meta_new)
labels = probs.argmax(axis=1)
print("Fused predictions:", labels)

216/216 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Fused predictions: [ 8  8 10 ... 10  4 10]


In [77]:
y_true = df_new['predicted_labels'].astype(int).values

In [78]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_true, final_labels)
print(f'Fused model accuracy: {accuracy:.2%}')

Fused model accuracy: 89.24%


# **Multi-Output Attention Fusion and Training**

In [103]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Synthetic Data/Best Final Predictions.csv')

# Determine unique label values across your tasks:
sphere_n = df['SPHERE Labels'].nunique()
mdd_n    = df['Predicted_mdd'].nunique()
stress_n = df['Predicted_Stress'].nunique()

print(sphere_n, mdd_n, stress_n)

20 2 2


In [104]:
C = max(sphere_n, mdd_n, stress_n)

In [105]:
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Input, Dense, Softmax, Multiply, Lambda
from tensorflow.keras.models import Model

# Define helper Lambda functions
def squeeze_fn(x): return K.squeeze(x, axis=-1)
def expand_fn(x): return K.expand_dims(x, axis=-1)
def fused_fn(x): return K.sum(x, axis=1)

n_classes = C  # number of classes for SPHERE, MDD, Stress (assumed same here)

inp = Input(shape=(3, n_classes), name='meta_input')
scores = Dense(1)(inp)
scores = Lambda(squeeze_fn, output_shape=lambda s: (s[0], s[1]))(scores)
weights = Softmax(axis=1)(scores)
weights_exp = Lambda(expand_fn, output_shape=lambda s: (s[0], s[1], 1))(weights)
weighted = Multiply()([inp, weights_exp])
fused_vec = Lambda(fused_fn, output_shape=lambda s: (s[0], s[2]))(weighted)

out_sphere = Dense(n_classes, activation='softmax', name='sphere_out')(fused_vec)
out_mdd    = Dense(n_classes, activation='softmax', name='mdd_out')(fused_vec)
out_str    = Dense(n_classes, activation='softmax', name='stress_out')(fused_vec)

multi_fusion = Model(inp, [out_sphere, out_mdd, out_str])
multi_fusion.compile(optimizer='adam',
                     loss='categorical_crossentropy',
                     metrics=['accuracy'])

In [106]:
from tensorflow.keras.utils import to_categorical

sphere = to_categorical(df['SPHERE Labels'], num_classes=n_classes)
mdd    = to_categorical(df['Predicted_mdd'], num_classes=n_classes)
stress = to_categorical(df['Predicted_Stress'], num_classes=n_classes)

y_train = [sphere, mdd, stress]

In [108]:
multi_fusion.compile(
    optimizer='adam',
    loss={
        'sphere_out': 'categorical_crossentropy',
        'mdd_out':    'categorical_crossentropy',
        'stress_out': 'categorical_crossentropy'
    },
    metrics={
        'sphere_out': 'accuracy',
        'mdd_out':    'accuracy',
        'stress_out': 'accuracy'
    }
)

In [109]:
multi_fusion.fit(
    X_meta, y_train,
    validation_split=0.1,
    epochs=30,
    batch_size=32,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True)]
)

Epoch 1/30
194/194 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - loss: 8.5744 - mdd_out_accuracy: 0.2794 - mdd_out_loss: 2.7731 - sphere_out_accuracy: 0.2978 - sphere_out_loss: 2.7747 - stress_out_accuracy: 0.0349 - stress_out_loss: 3.0266 - val_loss: 7.3622 - val_mdd_out_accuracy: 0.5957 - val_mdd_out_loss: 2.3606 - val_sphere_out_accuracy: 0.4464 - val_sphere_out_loss: 2.4915 - val_stress_out_accuracy: 0.7319 - val_stress_out_loss: 2.5187
Epoch 2/30
194/194 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 6.8595 - mdd_out_accuracy: 0.5342 - mdd_out_loss: 2.1777 - sphere_out_accuracy: 0.6396 - sphere_out_loss: 2.3179 - stress_out_accuracy: 0.7193 - stress_out_loss: 2.3639 - val_loss: 5.8340 - val_mdd_out_accuracy: 0.5971 - val_mdd_out_loss: 1.8418 - val_sphere_out_accuracy: 0.7304 - val_sphere_out_loss: 2.0547 - val_stress_out_accuracy: 0.7652 - val_stress_out_loss: 1.9515
Epoch 3/30
194/194 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 5.3758 - mdd_out_accuracy: 0.5672 - mdd_out_loss: 1.7102 - sphere_out_ac

In [110]:
sphere_pred, mdd_pred, stress_pred = multi_fusion.predict(X_meta_new)

sphere_lbl = sphere_pred.argmax(axis=1)
mdd_lbl    = mdd_pred.argmax(axis=1)
stress_lbl = stress_pred.argmax(axis=1)

216/216 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


In [111]:
multi_fusion.save('/content/Multi-Output_Attention_Fusion.h5')

In [112]:
multi_fusion.save('/content/Multi-Output_Attention_Fusion.keras')

# **Fusion Testing**



In [120]:
import tensorflow.keras.backend as K
import keras
from tensorflow.keras.models import load_model
from tensorflow.keras.saving import custom_object_scope

# Redefine the functions exactly as during training
@keras.saving.register_keras_serializable()
def squeeze_fn(x):
    return K.squeeze(x, axis=-1)

@keras.saving.register_keras_serializable()
def expand_fn(x):
    return K.expand_dims(x, axis=-1)

@keras.saving.register_keras_serializable()
def fused_fn(x):
    return K.sum(x, axis=1)

with custom_object_scope({
    'squeeze_fn': squeeze_fn,
    'expand_fn': expand_fn,
    'fused_fn': fused_fn
}):
    fusion = load_model('/content/Multi-Output_Attention_Fusion.h5', compile=False)

In [121]:
meta_probs = fusion.predict(X_meta_new)
out_sphere, out_mdd, out_stress = meta_probs  # depending on model outputs

216/216 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


In [174]:
df_test = pd.read_csv('/content/drive/MyDrive/Synthetic Data/Stress Fusion Data.csv')
sample = df.sample(n=1, random_state=42).reset_index(drop=True)

def slice_input(df, start, ts, feat):
    arr = df.iloc[:, start : start + ts * feat].values
    return arr.reshape(-1, ts, feat)

X1 = slice_input(df_test, start=0,   ts=30, feat=4)
X2 = slice_input(df_test, start=120, ts=10, feat=48)
X3 = slice_input(df_test, start=600, ts=20, feat=23)

# Load your three base models (if not already loaded)
model1 = load_model('/content/drive/MyDrive/Thesis Models/mdd_lstm.h5', compile=False)
model2 = load_model('/content/drive/MyDrive/Thesis Models/lstm_stress_dryad_model.h5', compile=False)
model3 = load_model('/content/drive/MyDrive/Thesis Models/rnn_model.h5', compile=False)

P1 = model1.predict(X1)
P2 = model2.predict(X2)
P3 = model3.predict(X3)

# Ensure all have the same class dimension
maxC = max(P1.shape[1], P2.shape[1], P3.shape[1])
def pad(arr, target):
    return arr if arr.shape[1] == target else np.pad(arr, ((0,0),(0,target-arr.shape[1])))
P1, P2, P3 = pad(P1, maxC), pad(P2, maxC), pad(P3, maxC)

X_meta = np.stack([P1, P2, P3], axis=1)

216/216 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step
216/216 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
216/216 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step


In [175]:
out_sphere, out_mdd, out_stress = fusion.predict(X_meta)
pred_sphere = np.argmax(out_sphere, axis=1)
pred_mdd    = np.argmax(out_mdd, axis=1)
pred_str    = np.argmax(out_stress, axis=1)

216/216 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [176]:
act_s   = sample['predicted_labels'].values[0]
act_m   = sample['Actual_mdd'].values[0]
act_str = sample['Actual_Stress'].values[0]

In [177]:
print("✅ Sample index:", sample.index[0])
print("Actual labels:    SPHERE =", act_s, " | MDD =", act_m, " | STRESS =", act_str)
print("Predicted labels: SPHERE =", np.argmax(out_sphere, axis = 1)[0], " | MDD =", np.argmax(out_mdd, axis = 1)[0], " | STRESS =", np.argmax(out_stress, axis = 1)[0])

✅ Sample index: 0
Actual labels:    SPHERE = 10  | MDD = 1  | STRESS = 1
Predicted labels: SPHERE = 8  | MDD = 1  | STRESS = 1


In [178]:
from sklearn.metrics import accuracy_score, classification_report

# Replace with your actual test label columns
y_true_sphere = df_test['predicted_labels'].astype(int).values
y_true_mdd    = df_test['Actual_mdd'].astype(int).values
y_true_str    = df_test['Actual_Stress'].astype(int).values

print("Fusion Model Report:\n", classification_report(y_true_sphere, pred_sphere))

Fusion Model Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00        21
           1       0.00      0.00      0.00        46
           2       0.50      0.04      0.08        23
           3       0.00      0.00      0.00         0
           4       0.88      0.75      0.81       669
           5       0.84      0.81      0.82       197
           6       1.00      0.53      0.69        78
           7       0.90      0.99      0.95       503
           8       0.94      0.99      0.97      1861
           9       1.00      0.41      0.58        22
          10       0.84      0.99      0.91      2829
          11       1.00      0.22      0.35        51
          12       0.00      0.00      0.00        28
          13       1.00      0.08      0.15        60
          14       0.92      0.58      0.71        60
          15       0.94      0.33      0.49        48
          16       0.73      0.29      0.41        28
     

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

# **Large Sample Testing**

In [182]:
import pandas as pd
import numpy as np
import tensorflow.keras.backend as K
import keras
from tensorflow.keras.models import load_model
from tensorflow.keras.saving import custom_object_scope

In [183]:
@keras.saving.register_keras_serializable()
def squeeze_fn(x): return K.squeeze(x, axis=-1)

@keras.saving.register_keras_serializable()
def expand_fn(x): return K.expand_dims(x, axis=-1)

@keras.saving.register_keras_serializable()
def fused_fn(x): return K.sum(x, axis=1)

with custom_object_scope({
    'squeeze_fn': squeeze_fn,
    'expand_fn': expand_fn,
    'fused_fn': fused_fn
}):
    fusion = load_model('/content/Multi-Output_Attention_Fusion.h5', compile=False)

In [184]:
model1 = load_model('/content/drive/MyDrive/Thesis Models/mdd_lstm.h5', compile=False)
model2 = load_model('/content/drive/MyDrive/Thesis Models/lstm_stress_dryad_model.h5', compile=False)
model3 = load_model('/content/drive/MyDrive/Thesis Models/rnn_model.h5', compile=False)

In [185]:
df = pd.read_csv('/content/drive/MyDrive/Synthetic Data/Stress Fusion Data.csv')
sample = df.sample(n=100).reset_index(drop=True)

In [186]:
def slice_input(df, start, ts, feat):
    arr = df.iloc[:, start:start + ts*feat].values
    return arr.reshape(-1, ts, feat)

X1 = slice_input(sample, 0,   30, 4)
X2 = slice_input(sample, 120, 10, 48)
X3 = slice_input(sample, 600, 20, 23)

# ---- 3. Predict sub-models ----
P1 = model1.predict(X1, batch_size=32, verbose=0)
P2 = model2.predict(X2, batch_size=32, verbose=0)
P3 = model3.predict(X3, batch_size=32, verbose=0)

# ---- 4. Align class dims and create meta tensor ----
maxC = max(P1.shape[1], P2.shape[1], P3.shape[1])
def pad(arr): return arr if arr.shape[1]==maxC else np.pad(arr, ((0,0),(0, maxC-arr.shape[1])))
P1, P2, P3 = pad(P1), pad(P2), pad(P3)
X_meta = np.stack([P1, P2, P3], axis=1)

In [187]:
out_sphere, out_mdd, out_stress = fusion.predict(X_meta, batch_size=32, verbose=0)
pred_s = np.argmax(out_sphere, axis=1)
pred_m = np.argmax(out_mdd, axis=1)
pred_t = np.argmax(out_stress, axis=1)

# ---- 6. Extract actual labels ----
act_s = sample['predicted_labels'].astype(int).values
act_m = sample['Actual_mdd'].astype(int).values
act_t = sample['Actual_Stress'].astype(int).values

In [188]:
results = sample.copy().loc[:, :]  # include all original columns
results = results.assign(
    sphere_actual   = act_s,
    mdd_actual      = act_m,
    stress_actual   = act_t,
    sphere_pred     = pred_s,
    mdd_pred        = pred_m,
    stress_pred     = pred_t
)

In [189]:
output_path = '/content/drive/MyDrive/Synthetic Data/Fusion_Stress_100_Final.csv'
results.to_csv(output_path, index=False)
print(f"✅ Saved 100‑sample results to:\n  {output_path}")

✅ Saved 100‑sample results to:
  /content/drive/MyDrive/Synthetic Data/Fusion_Stress_100_Final.csv


In [190]:
import pandas as pd
from sklearn.metrics import classification_report

# Load the CSV with actual and predicted columns
df = pd.read_csv('/content/drive/MyDrive/Synthetic Data/Fusion_Stress_100_Final.csv')

# Extract arrays for true / predicted labels
y_true_sphere = df['sphere_actual']
y_pred_sphere = df['sphere_pred']

y_true_mdd = df['mdd_actual']
y_pred_mdd = df['mdd_pred']

y_true_stress = df['stress_actual']
y_pred_stress = df['stress_pred']

# Print classification reports
print("==== SPHERE Classification Report ====")
print(classification_report(y_true_sphere, y_pred_sphere))

print("==== MDD Classification Report ====")
print(classification_report(y_true_mdd, y_pred_mdd))

print("==== STRESS Classification Report ====")
print(classification_report(y_true_stress, y_pred_stress))

==== SPHERE Classification Report ====
              precision    recall  f1-score   support

           4       1.00      1.00      1.00        10
           5       1.00      1.00      1.00         1
           7       0.78      1.00      0.88         7
           8       0.86      1.00      0.93        25
           9       1.00      1.00      1.00         1
          10       0.96      1.00      0.98        43
          11       1.00      1.00      1.00         1
          13       0.00      0.00      0.00         2
          15       1.00      1.00      1.00         1
          17       1.00      0.17      0.29         6
          18       1.00      1.00      1.00         1
          19       1.00      0.50      0.67         2

    accuracy                           0.92       100
   macro avg       0.88      0.81      0.81       100
weighted avg       0.91      0.92      0.89       100

==== MDD Classification Report ====
              precision    recall  f1-score   support

   

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m

# **OVERALL STRESS**

In [194]:
import pandas as pd

# Load your results (with predicted MDD and Stress)
df = pd.read_csv('/content/drive/MyDrive/Synthetic Data/Fusion_Stress_100_Final.csv')

# Compute predicted overall stress:
# If the average of Predicted_mdd and Predicted_Stress is ≥ 0.5 → 1, else 0
df['Predicted Overall Stress'] = (
    ((df['mdd_pred'] + df['stress_pred']) / 2) >= 0.5
).astype(int)

# If you also have the actual overall-stress labels:
print("Classification report for Overall Stress:")
from sklearn.metrics import classification_report
print(classification_report(df['Overall_Stress'], df['Predicted Overall Stress']))

# Save updated CSV
df.to_csv('/content/drive/MyDrive/Synthetic Data/Fusion_With_OverallStress.csv', index=False)
print("✅ Updated CSV saved with 'Predicted Overall Stress'.")

Classification report for Overall Stress:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        23
           1       0.77      1.00      0.87        77

    accuracy                           0.77       100
   macro avg       0.39      0.50      0.44       100
weighted avg       0.59      0.77      0.67       100

✅ Updated CSV saved with 'Predicted Overall Stress'.


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
